In [3]:
import pandas as pd
from litellm import completion
import litellm
import os
litellm.enable_json_schema_validation = True
litellm.set_verbose = True
from ollama import chat
from ollama import ChatResponse
import json
from litellm import ModelResponse
from dotenv import load_dotenv
load_dotenv()
# df = pd.read_json("hf://datasets/HuggingFaceH4/MATH-500/test.jsonl", lines=True)

True

In [4]:
import time
df = pd.read_csv('50 rows.csv')

In [5]:
df.head(2)

,Unnamed: 0.1,Unnamed: 0,problem,solution,answer,subject,level,unique_id,llm_raw_response,llm_answer,is_correct
0,0,0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,2,test/precalculus/807.json,"<think>\nOkay, so I need to convert the point ...","The point $(0,3)$ in rectangular coordinates c...",True
1,1,1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,5,test/intermediate_algebra/1994.json,"<think>\nOkay, so I have this problem where I ...",The given double sum can be rewritten by consi...,True


In [6]:
df.loc[0,'problem']

'Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta),$ where $r > 0$ and $0 \\le \\theta < 2 \\pi.$'

In [7]:
from pydantic import BaseModel
class Answer(BaseModel):
  reasoning: str
  answer: str

class AnswerCorrectness(BaseModel):
  correct:bool



In [7]:
# response: ChatResponse = chat(model='erwan2/DeepSeek-R1-Distill-Qwen-7B', messages=[
#             {
#                 'role': 'system',
#                 'content': '''Be a helpful assistant.
#                 You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output. 
                
#                 example:
#                 user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
#                 assistant output: area is 12 cm squared.
#                 ''',
#             },
#             {
#                 'role': 'user',
#                 'content': df.loc[0,'problem'],
#             }],
#         )
# answer_content = response['message']['content']
# thinking = answer_content[answer_content.find("<think>") + 7:answer_content.find("</think>")].strip()
# final_answer = answer_content[answer_content.find("</think>") + 8:].strip()
# answer_json = {
#     "reasoning": thinking,
#     "answer": final_answer
# }
# answer_obj = Answer.model_validate_json(json.dumps(answer_json))
# answer = answer_obj.answer

In [8]:
def process_math_problems(df, start_idx=0, end_idx=20):
    for idx in range(start_idx, min(len(df), end_idx + 1)):
        response:ChatResponse = chat(model='erwan2/DeepSeek-R1-Distill-Qwen-7B', messages=[
            {
                'role': 'system',
                'content': '''Be a helpful assistant.
                You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output. 
                
                example:
                user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
                assistant output: area is 12 cm squared.
                ''',
            },
            {
                'role': 'user',
                'content': df.iloc[idx]['problem'],
            }],
        )
        answer_content = response['message']['content']
        thinking = answer_content[answer_content.find("<think>") + 7:answer_content.find("</think>")].strip()
        final_answer = answer_content[answer_content.find("</think>") + 8:].strip()
        answer_json = {
            "reasoning": thinking,
            "answer": final_answer
        }
        answer_obj = Answer.model_validate_json(json.dumps(answer_json))
        answer = answer_obj.answer

        response = completion(model='gpt-4o', messages=[
            {
                'role': 'system',
                'content': '''You are an intellegent maths professor. I will give you 2 answers. 
                    One Model answer and one student answer. You whether the answer is right or wrong.Return a json with key are correct and value as True or False depeding on your evaluation''',
            },
            {
                'role': 'user',
                'content': f'Model Answer : {df.iloc[idx]["answer"]}, Student Answer : {answer}',
            },
        ], 
        response_format=AnswerCorrectness)
        
        answer_correctness = response.choices[0]['message']['content']
        answer_correctness_obj = AnswerCorrectness.model_validate_json(answer_correctness)
        
        df.at[idx, 'llm_raw_response'] = answer_content
        df.at[idx, 'llm_answer'] = answer
        df.at[idx, 'is_correct'] = answer_correctness_obj.correct
        
        print(f"Processed row {idx}")


In [9]:
process_math_problems(df,start_idx=43,end_idx=50)

19:48:12 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'correct': {'title': 'Correct', 'type': 'boolean'}}, 'required': ['correct'], 'title': 'AnswerCorrectness', 'type': 'object', 'additionalProperties': False}, 'name': 'AnswerCorrectness', 'strict': True}}, 'extra_body': {}}
RAW RESPONSE:
{"id": "chatcmpl-B07r3HIkm3aXWOTfFvNnsUieJLK8N", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"correct\":false}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739369893, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_50cad350e4", "usage": {"completion_tokens": 6, "prompt_tokens": 471, "total_tokens": 477, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio

19:48:34 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'correct': {'title': 'Correct', 'type': 'boolean'}}, 'required': ['correct'], 'title': 'AnswerCorrectness', 'type': 'object', 'additionalProperties': False}, 'name': 'AnswerCorrectness', 'strict': True}}, 'extra_body': {}}
RAW RESPONSE:
{"id": "chatcmpl-B07rPrDMDGFJl08arkBR7BxvrH6q7", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"correct\":true}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739369915, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_50cad350e4", "usage": {"completion_tokens": 6, "prompt_tokens": 169, "total_tokens": 175, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_

19:48:48 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


...

SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'correct': {'title': 'Correct', 'type': 'boolean'}}, 'required': ['correct'], 'title': 'AnswerCorrectness', 'type': 'object', 'additionalProperties': False}, 'name': 'AnswerCorrectness', 'strict': True}}, 'extra_body': {}}
RAW RESPONSE:
{"id": "chatcmpl-B07u2XmMxiwnTk3Az5o5VSaeTyPL6", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"correct\":true}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739370078, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_50cad350e4", "usage": {"completion_tokens": 6, "prompt_tokens": 143, "total_tokens": 149, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_

19:51:38 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'correct': {'title': 'Correct', 'type': 'boolean'}}, 'required': ['correct'], 'title': 'AnswerCorrectness', 'type': 'object', 'additionalProperties': False}, 'name': 'AnswerCorrectness', 'strict': True}}, 'extra_body': {}}
RAW RESPONSE:
{"id": "chatcmpl-B07uNmHmz22RDbHrl7WtJ28ocSpVJ", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"correct\":true}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739370099, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_4691090a87", "usage": {"completion_tokens": 6, "prompt_tokens": 124, "total_tokens": 130, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_

19:53:57 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'correct': {'title': 'Correct', 'type': 'boolean'}}, 'required': ['correct'], 'title': 'AnswerCorrectness', 'type': 'object', 'additionalProperties': False}, 'name': 'AnswerCorrectness', 'strict': True}}, 'extra_body': {}}
RAW RESPONSE:
{"id": "chatcmpl-B07wb8CaOulAZ3KBLXwyYcuPvH5KZ", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"correct\":false}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739370237, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_50cad350e4", "usage": {"completion_tokens": 6, "prompt_tokens": 130, "total_tokens": 136, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio

In [8]:
df.to_csv("50 rows.csv")

In [9]:
def generate_training_messages(df, start_idx=0, num_rows=20):
    end_idx = min(start_idx + num_rows, len(df))
    messages = [{
                'role': 'system',
                'content': '''Be a helpful assistant.
                You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output. 
                
                example:
                user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
                assistant output: area is 12 cm squared.'''
            }]
    for idx in range(start_idx, end_idx):
        row = df.iloc[idx]
        
        # Base messages that are common for all examples
        messages.extend([
            {
                'role': 'user',
                'content': row['problem']
            },
            {
                'role': 'assistant',
                'content': row['llm_raw_response']
            }
        ])
        
        # Add feedback based on correctness
        if not row['is_correct']:
            messages.append({
                'role': 'user',
                'content': f"Let me correct this. The right answer is {row['answer']}. Let's understand the solution: {row['solution']}"
            })
        else:
            messages.append({
                'role': 'user',
                'content': "Good job! Your reasoning and answer are correct!"
            })
        messages.append({
            'role': 'assistant',
            'content': "Understood. I will keep this in mind"
        })
            
       
    
    return messages

In [10]:
training_data = generate_training_messages(df)

In [11]:
len(training_data)

81

In [12]:
def test_process_math_problems_te(df, start_idx=20, end_idx=30):
    for idx in range(start_idx, min(len(df), end_idx + 1)):
        messages = training_data + [
            {
                'role': 'system',
                'content': '''Be a helpful assistant.
                You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output. 
                
                example:
                user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
                assistant output: area is 12 cm squared.''',
            },
            {
                'role': 'user',
                'content': df.iloc[idx]['problem'],
            }]
        
        response:ChatResponse = chat(model='erwan2/DeepSeek-R1-Distill-Qwen-7B', 
                                      messages=messages, options={"num_ctx": 20000})
        answer_content = response['message']['content']
        thinking = answer_content[answer_content.find("<think>") + 7:answer_content.find("</think>")].strip()
        final_answer = answer_content[answer_content.find("</think>") + 8:].strip()
        answer_json = {
            "reasoning": thinking,
            "answer": final_answer
        }
        answer_obj = Answer.model_validate_json(json.dumps(answer_json))
        answer = answer_obj.answer

        response = completion(model='gpt-4o', messages=[
            {
                'role': 'system',
                'content': '''You are an intellegent maths professor. I will give you 2 answers. 
                    One Model answer and one student answer. You whether the answer is right or wrong.Return a json with key are correct and value as True or False depeding on your evaluation''',
            },
            {
                'role': 'user',
                'content': f'Model Answer : {df.iloc[idx]["answer"]}, Student Answer : {answer}',
            },
        ], 
        response_format=AnswerCorrectness)
        
        answer_correctness = response.choices[0]['message']['content']
        answer_correctness_obj = AnswerCorrectness.model_validate_json(answer_correctness)
        
        df.at[idx, 'llm_raw_response_test'] = answer_content
        df.at[idx, 'llm_answer_test'] = answer
        df.at[idx, 'is_correct_test'] = answer_correctness_obj.correct
        
        print(f"Processed row {idx}")


In [18]:
test_process_math_problems_te(df,start_idx=20,end_idx=31)

02:49:01 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'correct': {'title': 'Correct', 'type': 'boolean'}}, 'required': ['correct'], 'title': 'AnswerCorrectness', 'type': 'object', 'additionalProperties': False}, 'name': 'AnswerCorrectness', 'strict': True}}, 'extra_body': {}}
RAW RESPONSE:
{"id": "chatcmpl-B0EQInFmIDsJoP1nUfROGTLnOK9S8", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"correct\":true}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739395142, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_4691090a87", "usage": {"completion_tokens": 6, "prompt_tokens": 255, "total_tokens": 261, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_

02:51:30 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'correct': {'title': 'Correct', 'type': 'boolean'}}, 'required': ['correct'], 'title': 'AnswerCorrectness', 'type': 'object', 'additionalProperties': False}, 'name': 'AnswerCorrectness', 'strict': True}}, 'extra_body': {}}
RAW RESPONSE:
{"id": "chatcmpl-B0ESg5iL4S3ZlpaxSMJOLL8xtTEOH", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"correct\":true}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739395290, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_4691090a87", "usage": {"completion_tokens": 6, "prompt_tokens": 426, "total_tokens": 432, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_

03:24:01 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


...

SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'correct': {'title': 'Correct', 'type': 'boolean'}}, 'required': ['correct'], 'title': 'AnswerCorrectness', 'type': 'object', 'additionalProperties': False}, 'name': 'AnswerCorrectness', 'strict': True}}, 'extra_body': {}}
RAW RESPONSE:
{"id": "chatcmpl-B0FaHQ5M25oPAcJM0KrfJOGgQn52i", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"correct\":false}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739399605, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_50cad350e4", "usage": {"completion_tokens": 6, "prompt_tokens": 504, "total_tokens": 510, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio

04:04:44 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'correct': {'title': 'Correct', 'type': 'boolean'}}, 'required': ['correct'], 'title': 'AnswerCorrectness', 'type': 'object', 'additionalProperties': False}, 'name': 'AnswerCorrectness', 'strict': True}}, 'extra_body': {}}
RAW RESPONSE:
{"id": "chatcmpl-B0FbZu0L1QiuZroc7z5uV1PhkBcl6", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"correct\":true}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739399685, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_4691090a87", "usage": {"completion_tokens": 6, "prompt_tokens": 915, "total_tokens": 921, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_

04:05:36 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'correct': {'title': 'Correct', 'type': 'boolean'}}, 'required': ['correct'], 'title': 'AnswerCorrectness', 'type': 'object', 'additionalProperties': False}, 'name': 'AnswerCorrectness', 'strict': True}}, 'extra_body': {}}
RAW RESPONSE:
{"id": "chatcmpl-B0FcPpfzRrstcxsNSyq8SdY80Tfm3", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"correct\":true}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739399737, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_4691090a87", "usage": {"completion_tokens": 6, "prompt_tokens": 562, "total_tokens": 568, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_

In [19]:
df.to_csv('first 50 test.csv')

In [20]:
df_test = df.iloc[20:50]

In [21]:
df_test['is_correct'].value_counts()

is_correct
True     25
False     5
Name: count, dtype: int64

In [22]:
df_test['is_correct_test'].value_counts()

is_correct_test
True     23
False     7
Name: count, dtype: int64

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.llms.anthropic import Anthropic
